In [1]:
import web3
import json

In [2]:
web3 = web3.Web3(web3.Web3.HTTPProvider("http://127.0.0.1:8545",request_kwargs={'timeout': 3600}))

# check if connection was successful
assert web3.isConnected()

In [3]:
!curl -X POST -H "Content-Type: application/json" --data '{"jsonrpc":"2.0","method":"web3_clientVersion","params":[],"id":67}' http://127.0.0.1:8545

{"jsonrpc":"2.0","id":67,"result":"Geth/v1.10.24-stable-972007a5/linux-amd64/go1.18.2"}


In [4]:
!pwd

/smartcode/notebooks


In [5]:
cd ../data

/smartcode/data


In [6]:
!pwd

/smartcode/data


In [7]:
!pwd

/smartcode/data


# EVM OPCODES

https://www.ethervm.io/

In [4]:
import os
from os import listdir
from os.path import isfile, join
from collections import defaultdict

In [5]:
block_path = "/smartcode/data/ethereum_traces/geth_dumps/blocks"
block_tracer_path = "/smartcode/data/ethereum_traces/geth_dumps/blocks_tracer"
output_path = "/smartcode/data/ethereum_traces/analysis"

In [6]:
context_opcodes = ["BLOCKHASH","COINBASE","TIMESTAMP","NUMBER","DIFFICULTY","GASLIMIT"]

In [7]:
state_opcodes = ["BALANCE","SELFBALANCE","BASEFEE"]

In [8]:
other_opcodes = ["CHAINID"]

In [9]:
import csv
evm_opcodes = open("evm_opcodes.csv", "r")

dict_reader = csv.DictReader(evm_opcodes)

print(dict_reader.fieldnames)
# dict to resolve opcode names to number
interested_opcodes_dict = {}
for row in dict_reader:
    interested_opcodes_dict[row["Name"]]=row


FileNotFoundError: [Errno 2] No such file or directory: 'evm_opcodes.csv'

In [10]:
print(interested_opcodes_dict["BLOCKHASH"])

NameError: name 'interested_opcodes_dict' is not defined

In [11]:
#for key in interested_opcodes_dict:
#    print(key)

In [18]:
interested_opcodes = context_opcodes + state_opcodes + other_opcodes

In [17]:
#testtxpath = "/smartcode/data/ethereum_traces/geth_dumps/0x3dac2080b4c423029fcc9c916bc430cde441badfe736fc6d1fe9325348af80fd.json"

In [18]:
#os.path.basename(testtxpath)

In [19]:
#with open(testtxpath) as datafile:
#    test_data = json.load(datafile)


In [20]:
#for key in test_data:
#    print(key)

gas
failed
returnValue
structLogs


In [21]:
#testanalysis_path = join(output_path,"0x2a306e34586c22563afc1488541ba86b321940cad4255b162c4ca61adb3c046c.json")

In [22]:
#with open(testanalysis_path) as datafile:
#    test_blockanalysis = json.load(datafile)


FileNotFoundError: [Errno 2] No such file or directory: '/smartcode/data/ethereum_traces/analysis/0x2a306e34586c22563afc1488541ba86b321940cad4255b162c4ca61adb3c046c.json'

# Tests with debug and trace transaction rpc calls

Read up on tracers here

https://geth.ethereum.org/docs/dapp/tracing-filtered

In [17]:
disableMemory = True
disableStack = True
disableStorage = True
timeout = 3600

In [18]:
test_tx_hash = "0x00be462ee0cd651422bc2d66445989e14821d089fb11a4d8203d522f65e96f61"

In [19]:
test_tx = web3.eth.get_transaction(test_tx_hash)
# useful for figuring out failed txns!
test_tx_receipt = web3.eth.get_transaction_receipt(test_tx_hash)
print(test_tx)
print(test_tx_receipt)

AttributeDict({'blockHash': HexBytes('0x2a306e34586c22563afc1488541ba86b321940cad4255b162c4ca61adb3c046c'), 'blockNumber': 14000292, 'from': '0xcA610B0783915cd8df283D2A6BfAe3FF29149Fdb', 'gas': 89943, 'gasPrice': 899932249376, 'maxFeePerGas': 1726892986452, 'maxPriorityFeePerGas': 2500000000, 'hash': HexBytes('0x00be462ee0cd651422bc2d66445989e14821d089fb11a4d8203d522f65e96f61'), 'input': '0xe674a0bd000000000000000000000000ca610b0783915cd8df283d2a6bfae3ff29149fdb0000000000000000000000000000000000000000000000000000000000000000', 'nonce': 398, 'to': '0xA68Dd8cB83097765263AdAD881Af6eeD479c4a33', 'transactionIndex': 28, 'value': 10000000000000000, 'type': '0x2', 'accessList': [], 'chainId': '0x1', 'v': 0, 'r': HexBytes('0xfb935814deba5ae1f88c7c9d2e04eb4be88e8a831a0cc94a437a99b8d04ee465'), 's': HexBytes('0x7ebadfeae13ee4a4f1c4852b66a8be5d5f1621b4cd4dcf15f757f04049e7c2fe')})
AttributeDict({'blockHash': HexBytes('0x2a306e34586c22563afc1488541ba86b321940cad4255b162c4ca61adb3c046c'), 'blockNumbe

In [376]:
tracer = '{data: [], fault: function(log) {}, step: function(log) { if(log.op.toString() == "CALL") this.data.push(log.stack.peek(0)); }, result: function() { return this.data; }}'

In [447]:
tracer = """ 
{
         retVal: [], step: function(log,db) {this.retVal.push(log.getPC() + ":" + log.op.toString())},
         fault: function(log,db) {this.retVal.push("FAULT: " + JSON.stringify(log))}, 
         result: function(ctx,db) {return this.retVal}
} 
      """

In [444]:
tracer = """
      {
         retVal: [],
         step: function(log,db) {
            if(log.op.toNumber() == 0x54)
              this.retVal.push( "{ pc: "+ log.getPC() + "op : SLOAD");
            if(log.op.toNumber() == 0x55) 
              this.retVal.push("pc: "+ log.getPC() + " op : SSTORE");
         },
         fault: function(log,db) {this.retVal.push("FAULT: " + JSON.stringify(log))},
         result: function(ctx,db) {return this.retVal}
      }
"""

In [451]:
tracer = """
      {
         retVal: [],
         step: function(log,db) {
            if(log.op.toNumber() == 0x40)
              this.retVal.push("BLOCKHASH");
            if(log.op.toNumber() == 0x43) 
              this.retVal.push(log.getPC() + ": NUMBER");
         },
         fault: function(log,db) {this.retVal.push("FAULT: " + JSON.stringify(log))},
         result: function(ctx,db) {return this.retVal}
      }
"""

In [17]:
for op in interested_opcodes:
    print(interested_opcodes_dict[op])

NameError: name 'interested_opcodes' is not defined

In [21]:
tracer = " { retVal: [], step: function(log,db) { "
for op in interested_opcodes:
    tracer += "if(log.op.toNumber() == 0x"+interested_opcodes_dict[op]["Stack"]+") "
    tracer += "this.retVal.push(\""+op+"\" ); "
tracer += """},
         fault: function(log,db) {this.retVal.push("FAULT: " + JSON.stringify(log))},
         result: function(ctx,db) {return this.retVal}
      }
      """
print(tracer)

 { retVal: [], step: function(log,db) { if(log.op.toNumber() == 0x40) this.retVal.push("BLOCKHASH" ); if(log.op.toNumber() == 0x41) this.retVal.push("COINBASE" ); if(log.op.toNumber() == 0x42) this.retVal.push("TIMESTAMP" ); if(log.op.toNumber() == 0x43) this.retVal.push("NUMBER" ); if(log.op.toNumber() == 0x44) this.retVal.push("DIFFICULTY" ); if(log.op.toNumber() == 0x45) this.retVal.push("GASLIMIT" ); if(log.op.toNumber() == 0x31) this.retVal.push("BALANCE" ); if(log.op.toNumber() == 0x47) this.retVal.push("SELFBALANCE" ); if(log.op.toNumber() == 0x48) this.retVal.push("BASEFEE" ); if(log.op.toNumber() == 0x46) this.retVal.push("CHAINID" ); },
         fault: function(log,db) {this.retVal.push("FAULT: " + JSON.stringify(log))},
         result: function(ctx,db) {return this.retVal}
      }
      


In [456]:
tx_trace = web3.manager.request_blocking("debug_traceTransaction",[test_tx_hash, {"disableStorage": disableStorage,"disableMemory": disableMemory,"disableStack": disableStack,  "timeout": str(timeout)+"s", "tracer": tracer,}])

In [457]:
for i in tx_trace:
    print(i)

TIMESTAMP


# Hacky code to analyze tx traces and dump stats on interested opcode occurence

In [19]:
def analyze_tx_trace(file_path,interested_opcodes):
    tx_summary_d = {}
    try:
        base = os.path.basename(file_path)
        root_ext = os.path.splitext(base)
        
        # if error also note this down
        if root_ext[1] == ".error":
            print("encountered error for tx {}".format(file_path))
            # its possible that we have both error and regular json files so make sure to not overwrite them!
            if not "hash" in tx_summary_d:
                tx_summary_d["hash"] = root_ext[0]
                tx_summary_d["error"] = True 
        else:
            tx_summary_d["hash"] = root_ext[0]
            with open(file_path) as datafile:
                data = json.load(datafile)
            #TODO our custom tracer does't include this yet - we can always fetch this from the tx summary from a block i guess?
            #tx_summary_d["failed"] = data["failed"]
  
        
            #parse opcodes and count ones we are interested in
            # note that is if is a dirty hack to allow for blocks parsed with a tracer to also be analyzed
            tx_opcodes_d = defaultdict(int)
            
            if "structLogs" in data:
                ops = data["structLogs"]
                for op in ops:
                    opcode = op["op"]
                    if opcode in interested_opcodes:
                        tx_opcodes_d[opcode] += 1
                        
                tx_summary_d["interested_opcodes"] = tx_opcodes_d

            else:
                ops = data
                for op in ops:
                    opcode = op
                    if opcode in interested_opcodes:
                        tx_opcodes_d[opcode] += 1
                        
                tx_summary_d["interested_opcodes"] = tx_opcodes_d

    except Exception as e:
        print(file_path,str(e))
        raise e 
    
    return tx_summary_d

In [20]:
#problem_file_path = "/smartcode/data/ethereum_traces/geth_dumps/blocks/0x1644f62f3a84c807569131fb5a320b583473264da2b2b8c0a82b17f1305bb3e0/0x98ba7f08a61c48d1604db9549e1351ffcb8ac03629590312ec11fb2a1185c979.json"

In [21]:
#analyze_tx_trace(problem_file_path ,interested_opcodes)

In [22]:
block_14m = web3.eth.get_block(14000000)

In [23]:
#print(block_14m)

In [24]:
print(block_14m["hash"].hex())

0x9bff49171de27924fa958faf7b7ce605c1ff0fdee86f4c0c74239e6ae20d9446


In [13]:
print(block_path)
print(block_tracer_path)

/smartcode/data/ethereum_traces/geth_dumps/blocks
/smartcode/data/ethereum_traces/geth_dumps/blocks_tracer


In [25]:
combined_results = {}

In [26]:
def process_blocks_from_path_list(_block_path, _combined_results):
    pathlist = listdir(_block_path)
    # check if we already have combined_results, if yes just update entries
    for path in pathlist:
        if not isfile(path):
            
            if path not in _combined_results:
                _combined_results[path] = {}
                _combined_results[path]["transactions"] = {}
                  
            block_txns_d = _combined_results[path]["transactions"]
            subdir = join(_block_path,path)

            # Single tx trace
            # analyze occurence of opcodes we are interested in
            for file in listdir(subdir):
                # figure out if we have tx_trace already processed
                base = os.path.basename(join(subdir,file))
                root_ext = os.path.splitext(base)
                
                # root_ext[0] is filename w.o extension e.g. tx_hash
                if root_ext[0] in block_txns_d:
                    if "error" not in block_txns_d[root_ext[0]]:
                        continue
                
                result = analyze_tx_trace(join(subdir,file),interested_opcodes)
                tx_hash = result["hash"]
                block_txns_d[tx_hash] = result
                
            
            #write results to files with blockhash as filename
            outputfile = join(join(output_path,"blocks"),path+".json")
            os.makedirs(os.path.dirname(outputfile), exist_ok=True)
            with open(outputfile , 'w', encoding='utf-8') as f:
                json.dump(block_txns_d,f)
    #dump total dict
    with open(join(output_path,"combined_dict.json" ), 'w', encoding='utf-8') as f:
        json.dump(_combined_results,f) 

In [27]:
#get combined results for all blocks we have:
#process_blocks_from_path_list(block_path,combined_results)

In [28]:
# add blocks from tracer path!
process_blocks_from_path_list(block_tracer_path,combined_results)

In [29]:
#write-back combined results
with open(join(output_path,"combined_dict.json" ), 'w', encoding='utf-8') as f:
    json.dump(combined_results,f) 

In [30]:
for key,value in combined_results.items():
    print(value)
    break

{'transactions': {'0x049e13a4e10dd8150747b6583e5d8345235db29b11ed2a2e8e67b54d41342181': {'hash': '0x049e13a4e10dd8150747b6583e5d8345235db29b11ed2a2e8e67b54d41342181', 'interested_opcodes': defaultdict(<class 'int'>, {})}, '0x082aec77d91fbeaec29ec8d18b8fa46ef725929d6caa2df3f61b991cb53bca8e': {'hash': '0x082aec77d91fbeaec29ec8d18b8fa46ef725929d6caa2df3f61b991cb53bca8e', 'interested_opcodes': defaultdict(<class 'int'>, {})}, '0x0b2b3c81315d0f1c8dc08823bd5c287cf6e16e4f8f278ce929c39d4dfd2c1b9a': {'hash': '0x0b2b3c81315d0f1c8dc08823bd5c287cf6e16e4f8f278ce929c39d4dfd2c1b9a', 'interested_opcodes': defaultdict(<class 'int'>, {})}, '0x0fcabe0cade9f38232803867a31c0ef0bed2a08892c0e13e15db7540f327e9c7': {'hash': '0x0fcabe0cade9f38232803867a31c0ef0bed2a08892c0e13e15db7540f327e9c7', 'interested_opcodes': defaultdict(<class 'int'>, {})}, '0x1013912a19b7d92a3517b5874eacfb8d1ea91ea3c150561833fb07e3200fba0f': {'hash': '0x1013912a19b7d92a3517b5874eacfb8d1ea91ea3c150561833fb07e3200fba0f', 'interested_opcod

In [31]:
#load combined results
with open(join(output_path,"combined_dict.json")) as datafile:
    combined_results = json.load(datafile)

In [32]:
def writeBlockMap(block_map):
    #write-back map once processed
    # todo convert web3 attribute dict to json?
    for key,value in block_map.items():
        block_map[key] = json.loads(web3.toJSON(block_map[key]))
    with open(join(output_path,"block_map.json" ), 'w', encoding='utf-8') as f:
        json.dump(block_map,f) 

In [33]:
# query archive node for block data for blocks we have

#load map of blockhash->height
status_update = 25
block_map = {}
if os.path.exists(join(output_path,"block_map.json")):
    with open(join(output_path,"block_map.json")) as datafile:
        block_map = json.load(datafile)
    
i = 0
for key in combined_results:
    if key not in block_map:
        block_map[key] = web3.eth.get_block(key)
        i+=1
        if i % status_update == 0 :
            print("updated {0} blocks".format(i))
            
    
writeBlockMap(block_map)

updated 25 blocks
updated 50 blocks
updated 75 blocks
updated 100 blocks
updated 125 blocks
updated 150 blocks
updated 175 blocks
updated 200 blocks
updated 225 blocks
updated 250 blocks
updated 275 blocks
updated 300 blocks
updated 325 blocks
updated 350 blocks
updated 375 blocks
updated 400 blocks
updated 425 blocks
updated 450 blocks
updated 475 blocks
updated 500 blocks
updated 525 blocks
updated 550 blocks
updated 575 blocks
updated 600 blocks
updated 625 blocks
updated 650 blocks
updated 675 blocks
updated 700 blocks
updated 725 blocks
updated 750 blocks
updated 775 blocks
updated 800 blocks
updated 825 blocks
updated 850 blocks
updated 875 blocks
updated 900 blocks
updated 925 blocks
updated 950 blocks
updated 975 blocks
updated 1000 blocks
updated 1025 blocks
updated 1050 blocks
updated 1075 blocks
updated 1100 blocks
updated 1125 blocks
updated 1150 blocks
updated 1175 blocks
updated 1200 blocks
updated 1225 blocks
updated 1250 blocks
updated 1275 blocks
updated 1300 blocks
up

In [34]:
def writeWeb3Dict(dictionary,filename):
    #write-back map once processed
    # todo convert web3 attribute dict to json?
    for key,value in dictionary.items():
        dictionary[key] = json.loads(web3.toJSON(dictionary[key]))
    with open(join(output_path,filename), 'w', encoding='utf-8') as f:
        json.dump(dictionary,f) 

In [35]:
def readWeb3Dict(filename):
    web3dict = {}
    
    if os.path.exists(join(output_path,filename)):
        with open(join(output_path,filename)) as datafile:
            web3dict = json.load(datafile)
    return web3dict
    

In [36]:
writeBlockMap(block_map)

In [37]:
def getBlockMap(block_map):
    if block_map is None:
        block_map = {}
        if os.path.exists(join(output_path,"block_map.json")):
            with open(join(output_path,"block_map.json")) as datafile:
                block_map = json.load(datafile)
    return block_map
            


In [38]:
#block_map=getBlockMap(None)

In [39]:
print(len(combined_results))

14075


In [132]:
print(combined_results["0xb91b09f29b8eb0f40b0a7944bb6087bb25d9aa77f229a3934ea8d30fd989e63c"])

{'transactions': {'0x003b156363f0f2982f3a9dfe7ae24e6ddaefb7f1f33746a2038b1e910f108a48': {'hash': '0x003b156363f0f2982f3a9dfe7ae24e6ddaefb7f1f33746a2038b1e910f108a48', 'interested_opcodes': defaultdict(<class 'int'>, {})}, '0x00b4827df585e4502b902bb95e2cb5a32baf9eb10c8566f89117ec762af22878': {'hash': '0x00b4827df585e4502b902bb95e2cb5a32baf9eb10c8566f89117ec762af22878', 'interested_opcodes': defaultdict(<class 'int'>, {})}, '0x010502bd83c70dfb72beb2e4ef62333fc1abaca2136d3147fef1b39aa216ee75': {'hash': '0x010502bd83c70dfb72beb2e4ef62333fc1abaca2136d3147fef1b39aa216ee75', 'interested_opcodes': defaultdict(<class 'int'>, {'TIMESTAMP': 3})}, '0x019425ae8fd52b5b2f4b55be498aebfe8b0021bea849057e4457584cbe3835d2': {'hash': '0x019425ae8fd52b5b2f4b55be498aebfe8b0021bea849057e4457584cbe3835d2', 'interested_opcodes': defaultdict(<class 'int'>, {})}, '0x02810a55dd62b45e0a823bfb35cf40e096d9ee4d23040d89d16701d29d19d46b': {'hash': '0x02810a55dd62b45e0a823bfb35cf40e096d9ee4d23040d89d16701d29d19d46b', 'in

In [61]:
print(list(combined_results.values())[0])

{'transactions': {'0x049e13a4e10dd8150747b6583e5d8345235db29b11ed2a2e8e67b54d41342181': {'hash': '0x049e13a4e10dd8150747b6583e5d8345235db29b11ed2a2e8e67b54d41342181', 'interested_opcodes': {}}, '0x082aec77d91fbeaec29ec8d18b8fa46ef725929d6caa2df3f61b991cb53bca8e': {'hash': '0x082aec77d91fbeaec29ec8d18b8fa46ef725929d6caa2df3f61b991cb53bca8e', 'interested_opcodes': {}}, '0x0b2b3c81315d0f1c8dc08823bd5c287cf6e16e4f8f278ce929c39d4dfd2c1b9a': {'hash': '0x0b2b3c81315d0f1c8dc08823bd5c287cf6e16e4f8f278ce929c39d4dfd2c1b9a', 'interested_opcodes': {}}, '0x0fcabe0cade9f38232803867a31c0ef0bed2a08892c0e13e15db7540f327e9c7': {'hash': '0x0fcabe0cade9f38232803867a31c0ef0bed2a08892c0e13e15db7540f327e9c7', 'interested_opcodes': {}}, '0x1013912a19b7d92a3517b5874eacfb8d1ea91ea3c150561833fb07e3200fba0f': {'hash': '0x1013912a19b7d92a3517b5874eacfb8d1ea91ea3c150561833fb07e3200fba0f', 'interested_opcodes': {}}, '0x176310cea95bd4ff8a3fc3d5de7af17e07eaa8cf1f273f9f4df222f7b9e74d0b': {'hash': '0x176310cea95bd4ff8a3f

In [62]:
print(list(block_map.values())[0]["gasLimit"])

29970705


In [40]:
# attach block height to combined_results
block_map = getBlockMap(block_map)
for key in combined_results:
    if "number" not in combined_results[key]:
        #first try to get it from our block map
        if key not in block_map:
            block_map[key] = web3.eth.get_block(key)
        combined_results[key]["number"] = block_map[key]["number"]
        

In [41]:
for value in combined_results.values():
    if "number" not in value:
        print(value)

In [42]:
print(list(combined_results.values())[0]["number"])

14021071


In [43]:
len(block_map)

15133

In [44]:
analysis_range = 8
#analysis_startheight = 14010000
analysis_startheight = 15510000
#analysis_startheight = 15550000

In [45]:
# determine empty blocks for the range we are intersted in
empty_block_dict = {}
for h in range (0,analysis_range+1):
    for j in range (0,10):
        for i in range(0,100):
            block_number = analysis_startheight+(h*10000)+(j*1000)+(i)
            block = None
            #probably better to define the range we are interested in rather than looping through...
            for value in block_map.values():
                if value["number"] == block_number:
                    block = value
                    break
            if block is None:        
                block = json.loads(web3.toJSON(web3.eth.get_block(block_number)))
            if block["hash"] not in block_map:
                block_map[block["hash"]] = block
            if len(block["transactions"]) == 0:
                print(block["hash"],block["number"])
                empty_block_dict[block["hash"]] = block


0x631f2334c9f6ce8d3e775157192989841f9a54d28a64d0758357aba1b19fe9ba 15510015
0xee2c68ab7d75276300cce57b0a7da8f8770559c8e71f4f486b3b2610c170da7f 15510028
0xeef372d89f21a1cddbc18bfd91683982b6096eb512a58b4fa5bd1bde7227e9aa 15510038
0x01eadc169917fd192a36dc1fdbbfe29cb627d7af93bacf895edaa61b54d63747 15510054
0x099a74865d7eb31ad934f7d5f44e85c57996f4d8ab30d660a6680a670ed0962b 15510056
0x9fb39b5980123d9361c337ec9285d4c87507ea370411f8a6b2c26283d6930ff9 15510064
0x32db905d206c838443052b2df8eedae66e42375fd22dac7b26b1bacfc5854d57 15511008
0x51e7144f128c34dec4fea2c1611ed9c2178b08d5be8537d888435c29f6554e0f 15511018
0xd46ad866808467ed6aede8f086734ec23d1a4df4eb3c017845271e8dc5416bb6 15511019
0x90602c78ae05d5231d8d42c460010c65f522ff0ae0050610d787d780520bf78b 15511032
0x7b5770e3bc351c422e8419948ad3847202597a091855b25fe28ae2a6f20a97b4 15511035
0x86a7c83c9dc4eec1e919f31fe3cf194a2133db8494eacd6817d4ad9180a785ba 15511059
0x82b4b372ba263fa55ab1f45465acf8d121bfa4986dbd9cd7797e07f21446c99e 15511088
0x892dc6fa7d

0x1c5b2be6695fb9aee991b40d4d06733e6c3ba26b62d44cc406eb3db739be9c12 15543036
0x17720faed811897c6cb6cfe0b65d88d905fab8ceb596b6b424b485b359579096 15544069
0x7a619ce512cbe45fe4513fae8857d495ff832b54aa45a248f0b1dfd8c158092d 15544083
0x740236004903ab8e8a369c71936576c6e35aa87a602965a1d87041d25f7ebdf8 15546009
0x224656c70c0039aa843faccc1ddfd2761328aa67e7c2525945e94ce685d20855 15546094
0x6e893038857ba874301f9884ce793db0ba3a5e5d98941dab7c6cf7ecc9dd86ce 15547012
0x9bcc101ccb9870c9553f54dd9418fd7ec021fc9deb1332e35ae802f6439deb0c 15547089
0xc365abea4aba9bdbc369c006b6febcba40c38bddbba707b149ae57abc27f56f3 15548043
0x07edec1fe5f9a507cab27d28a383b0e293260102daae2d8df00ca7c0745bcc73 15549000
0xb6c686cf1087726ddfe116ffde7dffd39808f88c48956bc4905ee8c5006cb136 15552079
0x8779add7707e126b7ecbf44dcfcbcca08f534f2b67c606f88dca8db4784589f4 15553033
0xef3dc8461b8b9e507c2c0768b08f56321a3d64f47bc853c80c6465373aec3bf8 15556038
0x493e333b3325a54976692a71b08a5b87fe97733cb1674152004582e808eabad9 15557010
0x8b252437d4

In [46]:
print(len(empty_block_dict))

143


In [47]:
# Note that we may have EMPTY blocks and they are NOT in our data!!!!
# filter out blocks relevant for our analysis
# blocks from 14010000 in 100 batches x 10, i.e. 14010000 - 14010099, etc.
analysis_dict = {}
# TODO more logical param


for key,value in combined_results.items():
    # range is exclusive of second param
    for i in range (0,analysis_range+1):
        for j in range (0,10):
            if value["number"] >= analysis_startheight+(i*10000)+(j*1000) and value["number"] <= analysis_startheight+(i*10000)+(j*1000)+99:
                analysis_dict[key]=value
                
    

In [182]:
# temporary code for dumping block traces -> expand our analysis to 2000 blocks ...
for j in range (0,10):
    cmd = "python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir geth_dumps/blocks_tracer --timeout 3600"
    #cmd+= " --blockRange "+str(14010000+(j*1000)+100)+" "+ str(14010099+(j*1000)+(1000-100))
    #cmd+= " --blockRange "+str(14010000+(j*1000))+" "+ str(14010000+(j*1000)+(100))+"; "
    #cmd+= " --blockRange "+str(14020000+(j*1000))+" "+ str(14020000+(j*1000)+(100))+"; "
    #cmd+= " --blockRange "+str(14030000+(j*1000))+" "+ str(14030000+(j*1000)+(100))+"; "
    cmd+= " --blockRange "+str(14040000+(j*1000))+" "+ str(14040000+(j*1000)+(100))+"; "
    #cmd+= " --blockRange "+str(14050000+(j*1000))+" "+ str(14050000+(j*1000)+(100))+"; "
    print(cmd)

python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir geth_dumps/blocks_tracer --timeout 3600 --blockRange 14040000 14040100; 
python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir geth_dumps/blocks_tracer --timeout 3600 --blockRange 14041000 14041100; 
python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir geth_dumps/blocks_tracer --timeout 3600 --blockRange 14042000 14042100; 
python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir geth_dumps/blocks_tracer --timeout 3600 --blockRange 14043000 14043100; 
python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir geth_dumps/blocks_tracer --timeout 3600 --blockRange 14044000 14044100; 
python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir

In [32]:
# temporary code for dumping block traces -> expand our analysis to 2000 blocks ...
cmd = ""
cmd1 = "python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir geth_dumps/blocks_tracer --timeout 3600"
for j in range (0,10):
       # cmd+= " --blockRange "+str(14010000+(j*1000)+100)+" "+ str(14010099+(j*1000)+(1000-100))
    #cmd+= " --blockRange "+str(15550000+(j*1000))+" "+ str(15550000+(j*1000)+(100))+"; "
    cmd+= cmd1+" --blockRange "+str(15560000+(j*1000))+" "+ str(15560000+(j*1000)+(100))+"; "
    cmd+= cmd1+" --blockRange "+str(15570000+(j*1000))+" "+ str(15570000+(j*1000)+(100))+"; "
    cmd+= cmd1+" --blockRange "+str(15580000+(j*1000))+" "+ str(15580000+(j*1000)+(100))+"; "
    cmd+= cmd1+" --blockRange "+str(15590000+(j*1000))+" "+ str(15590000+(j*1000)+(100))+"; "
print(cmd)

python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir geth_dumps/blocks_tracer --timeout 3600 --blockRange 15560000 15560100; python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir geth_dumps/blocks_tracer --timeout 3600 --blockRange 15570000 15570100; python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir geth_dumps/blocks_tracer --timeout 3600 --blockRange 15580000 15580100; python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir geth_dumps/blocks_tracer --timeout 3600 --blockRange 15590000 15590100; python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir geth_dumps/blocks_tracer --timeout 3600 --blockRange 15561000 15561100; python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir geth

In [48]:
len(analysis_dict)

8857

In [116]:
# do we have all blocks?
count = defaultdict(int)
for key,value in block_map.items():
    
    for i in range (0,10):
        if value["number"] >= (15540000+i*1000) and value["number"] <= (15540000+i*1000)+100:
            count[i] +=1
print(count)

    

defaultdict(<class 'int'>, {6: 101, 9: 101, 1: 101, 3: 101, 0: 101, 2: 101, 4: 101, 5: 101, 8: 101, 7: 101})


In [49]:
def getTotalBlocks():
    return len(empty_block_dict)+len(analysis_dict)

In [50]:
def getNonemptyBlocks():
    return getTotalBlocks()-len(empty_block_dict)

In [51]:
print("total blocks: "+str(getTotalBlocks()))
print("nonempty blocks: "+str(getNonemptyBlocks()))


total blocks: 9000
nonempty blocks: 8857


In [52]:
print(len(empty_block_dict)/getTotalBlocks())
print("{}".format(round((len(empty_block_dict)/getTotalBlocks()) * 100, 3) ))

0.01588888888888889
1.589


In [121]:
# stuff for figuring out error_Txns

In [122]:
#figure out the largest transaction we processed in terms of gas
#max_gas = 0
#for block_hash,block in analysis_dict.items():
#    for tx_hash,value in block["transactions"].items():
#        tx = web3.eth.get_transaction(tx_hash)
#        if tx["gas"] > max_gas:
#            max_gas = tx["gas"]
#            print(max_gas, tx["hash"].hex())


# Data Sanity Checks

Ensure that the data we are looking at is consistent

note that block_map contains the data for blocks queried from the full node, so is the ground truth to which we compare

In [192]:
#print(list(analysis_dict.values())[0]["transactions"])
#print(list(block_map.values())[0]["transactions"])

In [123]:
# make sure that for all blocks we are looking at the number of transactions we have are correct

for key in analysis_dict:
    assert(len(block_map[key]["transactions"]) == len(analysis_dict[key]["transactions"]))
    for tx_hash in block_map[key]["transactions"]:
        assert tx_hash in analysis_dict[key]["transactions"]


In [194]:
#len(list(block_map.values())[0]["transactions"])

In [124]:

# assert that we don't have error transactions
error_txns = set()

for block_hash,block in analysis_dict.items():
    for tx_hash,value in block["transactions"].items():
        if "error" in value:
            error_txns.add(tx_hash)
# make sure we have no error txns
for i in error_txns:
    print(i)
assert len(error_txns) == 0
        


In [125]:
#error_tx_dict = {}
#for error_tx in error_txns:
#    error_tx_dict[error_tx] = web3.eth.get_transaction(error_tx)


In [126]:
bashable_cmds = []
for value in error_tx_dict.values():
    print(value["gas"], "\t",value["hash"].hex())
                    #TODO just create a hacky output that generates a bash chained command for missing txns....
    cmd = "python3 extract_traces.py --ipcFile  geth.ipc --disableStorage --disableMemory --disableStack --outputDir geth_dumps/blocks --timeout 100000  --fixErrorTx "
    cmd += "/home/nicholas/geth_dumps/blocks/"+value["blockHash"].hex()+"/"+value["hash"].hex()+".error"
    cmd += ";  "
    
    bashable_cmds.append(cmd)
    
    

NameError: name 'error_tx_dict' is not defined

In [ ]:
#print(list(error_tx_dict.values())[0])

In [ ]:
#for i in bashable_cmds:
#    print(i)

In [53]:
# Create a dict where key = blockhash and value is a list of transactions we are interested in

#opcodes = ["BLOCKHASH","COINBASE","DIFFICULTY"]
opcodes = interested_opcodes
#this is a crap name as its a dict containing blocks containing transactions...
relevant_txns = {}

for key,block in analysis_dict.items():
    for transaction in block["transactions"].values():
        if "interested_opcodes" in transaction:
            for op in opcodes:
                if op in transaction["interested_opcodes"]:
                    if key not in relevant_txns:
                        relevant_txns[key] = []
                    relevant_txns[key].append(transaction)
                    break


In [54]:
len(relevant_txns)

8840

In [55]:
# number of transactions we processed
def getTotalTxcount():
    total_txns = 0
    for block in analysis_dict.values():
        total_txns += len(block["transactions"])
    return total_txns

In [56]:
print(getTotalTxcount())

1526679


In [57]:
relevant_tx_count = 0
for block in relevant_txns.values():
    for tx in block:
        relevant_tx_count += 1
print(relevant_tx_count)

541815


In [58]:
print("Percentage of txns that contain at least one opcode we are interested in: "+ str(relevant_tx_count/getTotalTxcount()))

Percentage of txns that contain at least one opcode we are interested in: 0.3548977879436345


Count occurences of different opcodes in Txns
Note: we only count if the opcode is present in a tx trace, not how often it is used!

In [59]:
# probabilities for intersted_opcode occuring in a transaction

opcode_probability_tx = defaultdict(float)
for key,block in relevant_txns.items():
    for tx in block:
        for op in tx["interested_opcodes"]:
            opcode_probability_tx[op] +=1
    
#for key,value in opcode_probability_tx.items():
 #   opcode_probability_tx[key] /= getTotalTxcount()
    
print(opcode_probability_tx)

defaultdict(<class 'float'>, {'TIMESTAMP': 488464.0, 'SELFBALANCE': 127392.0, 'CHAINID': 90308.0, 'NUMBER': 67190.0, 'BLOCKHASH': 8636.0, 'BASEFEE': 5676.0, 'COINBASE': 6152.0, 'BALANCE': 12724.0, 'DIFFICULTY': 8991.0, 'GASLIMIT': 3380.0})


In [60]:
# to what does it actually round?
count_p = 0.0
count = 0
for key in opcode_probability_tx:
    count += opcode_probability_tx[key]
    count_p += round(opcode_probability_tx[key]/getTotalTxcount() * 100, 3) 
print(count_p)
print(count/getTotalTxcount())

53.638999999999996
0.5364015618214438


In [61]:
opcodes_blocks = {}
for key,block in relevant_txns.items():
    blockdata = {}
    opcode_occurences = defaultdict(int)
    for tx in block:
        for op in tx["interested_opcodes"]:
            opcode_occurences[op] +=1
    blockdata["opcodes"] = opcode_occurences
    # grab all tx count from 
    blockdata["txcount"] = len(analysis_dict[key])
    opcodes_blocks[key] = blockdata
    

In [62]:
len(opcodes_blocks)

8840

In [63]:
print(list(opcodes_blocks.values())[0])

{'opcodes': defaultdict(<class 'int'>, {'TIMESTAMP': 57, 'SELFBALANCE': 15, 'CHAINID': 9, 'NUMBER': 6, 'BLOCKHASH': 1, 'BASEFEE': 1, 'COINBASE': 1, 'BALANCE': 1}), 'txcount': 2}


In [64]:
#figure out how many blocks contain at least one such opcode
opcode_block_occurrance = defaultdict(int)

for op in interested_opcodes:
    opcode_block_occurrance[op] += 0

for block in opcodes_blocks.values():
    for key,value in block["opcodes"].items():
        opcode_block_occurrance[key] += 1
        

    
print(opcode_block_occurrance)
for key,value in opcode_block_occurrance.items():
    print(key, float(value) / getTotalBlocks() )
    
    
    
    

defaultdict(<class 'int'>, {'BLOCKHASH': 3837, 'COINBASE': 2579, 'TIMESTAMP': 8831, 'NUMBER': 8256, 'DIFFICULTY': 1758, 'GASLIMIT': 1579, 'BALANCE': 5466, 'SELFBALANCE': 8677, 'BASEFEE': 3389, 'CHAINID': 8494})
BLOCKHASH 0.42633333333333334
COINBASE 0.28655555555555556
TIMESTAMP 0.9812222222222222
NUMBER 0.9173333333333333
DIFFICULTY 0.19533333333333333
GASLIMIT 0.17544444444444443
BALANCE 0.6073333333333333
SELFBALANCE 0.9641111111111111
BASEFEE 0.37655555555555553
CHAINID 0.9437777777777778


In [139]:
# Make sanity check for this interesting insight that blockhash is in almost every 3rd block!!!

In [65]:
counter_block_blockhash = 0
for block in analysis_dict.values():
    for tx in block["transactions"].values():
        if "BLOCKHASH" in tx["interested_opcodes"]:
            counter_block_blockhash +=1
            break
print(counter_block_blockhash)

3837


In [66]:
#make combined dict for printing table -> ensures keys, i.e. opcodes, are aligned (we could also order them i guess?)
sorted_opcode_list = ['TIMESTAMP','SELFBALANCE',
                      'CHAINID','NUMBER',
                      'BLOCKHASH','BASEFEE',
                      'COINBASE','BALANCE',
                      'DIFFICULTY','GASLIMIT']
opcode_table_dict = {}
for key in sorted_opcode_list:
    opcode_table_dict[key] = {}
    opcode_table_dict[key]["tx"] = opcode_probability_tx[key]
    opcode_table_dict[key]["block"] = opcode_block_occurrance[key] 
    

# Print a goddamn Table

In [67]:
table = """
\\begin{table}[!htpb]
%\\footnotesize

\\renewcommand{\\arraystretch}{1.3} % increase space between lines
\\newcommand{\\neigung}{33}

%\\setlength\\tabcolsep{1.5em}
\\scriptsize
\\begin{tabular}{p{2,5cm}"""
for key in opcode_table_dict:
    table+="|l"
table+="} \n"

table += """Opcode (OP) & """
for key in opcode_table_dict:
    table += "\\begin{rotate}{\\neigung} \\texttt{ "+key+ " }  \\end{rotate} & "
# remove last ampersand
table = table[:-2]

table += """\\\\ \\hline
"""

table += "TX containing OP & "
## fill in actual data

for key in opcode_table_dict:
    table += """{} & """.format(round(opcode_table_dict[key]["tx"]) )

# remove last ampersand
table = table[:-2]

table += """\\\\ \\hline
"""

table += "Pr() OP in TX & "
## fill in actual data

for key in opcode_table_dict:
    table += """{}\\% & """.format(round(opcode_table_dict[key]["tx"]/getTotalTxcount() * 100, 3) )

# remove last ampersand
table = table[:-2]

table += """\\\\ \\hline
"""

table += "Blocks cont. OP & "
## fill in actual data

for key in opcode_table_dict:
    table += """{} & """.format(round(opcode_table_dict[key]["block"]) )

# remove last ampersand
table = table[:-2]

table += """\\\\ \\hline
"""

table += "Pr() OP in Block & "
## fill in actual data

for key in opcode_table_dict:
    table += """{}\\% & """.format(round(opcode_table_dict[key]["block"]/getTotalBlocks() * 100, 2) )

# remove last ampersand
table = table[:-2]


table += """
\\end{tabular}
\\caption{Opcode occurrance in analyzed block range.}
\\label{tab:opcode}
\\end{table}
"""

print(table)


\begin{table}[!htpb]
%\footnotesize

\renewcommand{\arraystretch}{1.3} % increase space between lines
\newcommand{\neigung}{33}

%\setlength\tabcolsep{1.5em}
\scriptsize
\begin{tabular}{p{2,5cm}|l|l|l|l|l|l|l|l|l|l} 
Opcode (OP) & \begin{rotate}{\neigung} \texttt{ TIMESTAMP }  \end{rotate} & \begin{rotate}{\neigung} \texttt{ SELFBALANCE }  \end{rotate} & \begin{rotate}{\neigung} \texttt{ CHAINID }  \end{rotate} & \begin{rotate}{\neigung} \texttt{ NUMBER }  \end{rotate} & \begin{rotate}{\neigung} \texttt{ BLOCKHASH }  \end{rotate} & \begin{rotate}{\neigung} \texttt{ BASEFEE }  \end{rotate} & \begin{rotate}{\neigung} \texttt{ COINBASE }  \end{rotate} & \begin{rotate}{\neigung} \texttt{ BALANCE }  \end{rotate} & \begin{rotate}{\neigung} \texttt{ DIFFICULTY }  \end{rotate} & \begin{rotate}{\neigung} \texttt{ GASLIMIT }  \end{rotate} \\ \hline
TX containing OP & 488464 & 127392 & 90308 & 67190 & 8636 & 5676 & 6152 & 12724 & 8991 & 3380 \\ \hline
Pr() OP in TX & 31.995\% & 8.344\% & 5.915\%

In [68]:
# probabilities for combination of Number and Blockhash and number and coinbase
import collections
import itertools as it
opcodes_probability_tx = defaultdict(float)

for key,block in relevant_txns.items():
    for tx in block:
        combinations = list(it.combinations(interested_opcodes,2))
        for n_tuple in combinations:
            for i in n_tuple:
                if i not in tx["interested_opcodes"]:
                    break
            # only executes if no break in for
            else:
                opcodes_probability_tx[str(n_tuple)] +=1 
            

for key,value in opcodes_probability_tx.items():   
    opcodes_probability_tx[key] /= getTotalTxcount()


print("Probability of opcodes occuring in same transaction")
for key,value in  opcodes_probability_tx.items():
    print(key,value)

Probability of opcodes occuring in same transaction
('TIMESTAMP', 'SELFBALANCE') 0.07268129056599325
('TIMESTAMP', 'CHAINID') 0.05332948183606377
('BLOCKHASH', 'NUMBER') 0.0050717930881344405
('COINBASE', 'TIMESTAMP') 0.003951059784014845
('COINBASE', 'SELFBALANCE') 0.00262530630211066
('COINBASE', 'BASEFEE') 0.00041135038865406547
('TIMESTAMP', 'BASEFEE') 0.0035043385020688696
('SELFBALANCE', 'BASEFEE') 0.0007434437756725546
('TIMESTAMP', 'NUMBER') 0.029398452457916825
('NUMBER', 'SELFBALANCE') 0.011485060055191693
('SELFBALANCE', 'CHAINID') 0.010797292685626776
('COINBASE', 'NUMBER') 0.0014528266911380846
('BLOCKHASH', 'COINBASE') 0.00019912502890260495
('BLOCKHASH', 'TIMESTAMP') 0.0033608898792739012
('NUMBER', 'CHAINID') 0.0025132984733529447
('NUMBER', 'BASEFEE') 0.002461552166499965
('BASEFEE', 'CHAINID') 0.00019519492964794826
('TIMESTAMP', 'BALANCE') 0.003812196277016976
('BALANCE', 'SELFBALANCE') 0.0031335991390462566
('NUMBER', 'BALANCE') 0.0019879752063138353
('TIMESTAMP', '

In [69]:
# combination without repetition 
import itertools as it
list(it.combinations(interested_opcodes,2))

[('BLOCKHASH', 'COINBASE'),
 ('BLOCKHASH', 'TIMESTAMP'),
 ('BLOCKHASH', 'NUMBER'),
 ('BLOCKHASH', 'DIFFICULTY'),
 ('BLOCKHASH', 'GASLIMIT'),
 ('BLOCKHASH', 'BALANCE'),
 ('BLOCKHASH', 'SELFBALANCE'),
 ('BLOCKHASH', 'BASEFEE'),
 ('BLOCKHASH', 'CHAINID'),
 ('COINBASE', 'TIMESTAMP'),
 ('COINBASE', 'NUMBER'),
 ('COINBASE', 'DIFFICULTY'),
 ('COINBASE', 'GASLIMIT'),
 ('COINBASE', 'BALANCE'),
 ('COINBASE', 'SELFBALANCE'),
 ('COINBASE', 'BASEFEE'),
 ('COINBASE', 'CHAINID'),
 ('TIMESTAMP', 'NUMBER'),
 ('TIMESTAMP', 'DIFFICULTY'),
 ('TIMESTAMP', 'GASLIMIT'),
 ('TIMESTAMP', 'BALANCE'),
 ('TIMESTAMP', 'SELFBALANCE'),
 ('TIMESTAMP', 'BASEFEE'),
 ('TIMESTAMP', 'CHAINID'),
 ('NUMBER', 'DIFFICULTY'),
 ('NUMBER', 'GASLIMIT'),
 ('NUMBER', 'BALANCE'),
 ('NUMBER', 'SELFBALANCE'),
 ('NUMBER', 'BASEFEE'),
 ('NUMBER', 'CHAINID'),
 ('DIFFICULTY', 'GASLIMIT'),
 ('DIFFICULTY', 'BALANCE'),
 ('DIFFICULTY', 'SELFBALANCE'),
 ('DIFFICULTY', 'BASEFEE'),
 ('DIFFICULTY', 'CHAINID'),
 ('GASLIMIT', 'BALANCE'),
 ('GASLIMIT

In [70]:
#create dicts if we haven't got them yet - 
# note: we basically just cache node queries so this is independent of our opal analysis so no need to reset
if web3_txns is None:
    web3_txns = {}
if web3_txns_receipts is None:
    web3_txns_receipts = {}

NameError: name 'web3_txns' is not defined

In [71]:
web3_txns = readWeb3Dict("web3_transaction_cache.json")
web3_txns_receipts = readWeb3Dict("web3_transaction_receipts_cache.json")

In [72]:
writeWeb3Dict(web3_txns,"web3_transaction_cache.json")
writeWeb3Dict(web3_txns_receipts,"web3_transaction_receipts_cache.json")

In [73]:
# this is analysis specific
opal_like_transactions = []

for key,block in relevant_txns.items():
    for tx in block:
        if "BLOCKHASH" in tx["interested_opcodes"] and "NUMBER" in tx["interested_opcodes"]:
            opal_like_transactions.append(tx)
            # cache requests
            if tx["hash"] not in web3_txns:
                tx_data = web3.eth.get_transaction(tx["hash"])
                web3_txns[tx["hash"]] = tx_data
            if tx["hash"] not in web3_txns_receipts:
                tx_receipt =  web3.eth.get_transaction_receipt(tx["hash"])
                web3_txns_receipts[tx["hash"]] = tx_receipt

writeWeb3Dict(web3_txns,"web3_transaction_cache.json")
writeWeb3Dict(web3_txns_receipts,"web3_transaction_receipts_cache.json")               

In [74]:
print(len(opal_like_transactions))

7743


In [75]:
print("Percentage of txns that use Blockhash + Number opcodes (opal-like): "+ str(len(opal_like_transactions)/getTotalTxcount()))

Percentage of txns that use Blockhash + Number opcodes (opal-like): 0.0050717930881344405


In [76]:
# How many of the blocks we look at contain an opal-like transaction?
# yeah yeah we could probably do this much more elegantly....
blocks_with_opallike = 0
for block in analysis_dict.values():
    for tx in opal_like_transactions:
        if tx["hash"] in block["transactions"]:
            blocks_with_opallike += 1
            break
print(blocks_with_opallike)

3514


In [77]:
print(str(round(float(blocks_with_opallike)/ getTotalBlocks(),2) ))

0.39


In [78]:
# figure out how many of our opal-like transactions failed
num_tx_failed = 0
tx_failed_contract_calls = defaultdict(int)
for tx in opal_like_transactions:
    if web3_txns_receipts[tx["hash"]]['status'] == 0:
        num_tx_failed +=1
        tx_failed_contract_calls[web3_txns_receipts[tx["hash"]]["to"]] +=1
        print(web3_txns_receipts[tx["hash"]]["transactionHash"],web3_txns_receipts[tx["hash"]]["to"])
print(num_tx_failed)
print(tx_failed_contract_calls)

0xa991a189d423866ae4083d871f541ecbeed567d3ad5f8ab0891454598a871802 0xbbc53022Af15Bb973AD906577c84784c47C14371
0x0e229293889d014abdb6e571aad15b490ca280db733ddcb8f3002193cb6320bf 0x0000000099cB7fC48a935BcEb9f05BbaE54e8987
0x6d7be7c236b5a7115d9fd0e4ce203cc5215eae148d288e85274ef9c39581ccbb 0x0000000099cB7fC48a935BcEb9f05BbaE54e8987
0x97a2579134a43eb82d722ca7d6fec97fdaf31e8340d330eccd1b05ff9c5035cf 0x000000000035B5e5ad9019092C665357240f594e
0xe66006c96ee100cf0df0a3d80ad077669836241a88ece4ced12441c467f6f5ed 0x000000000035B5e5ad9019092C665357240f594e
0x8575db3e85ad8bd7be048e6c6964865d20b40f8fd152848eedb30e67629777d7 0xdb7A53E6AE058E1Dcf4502341E2ADFA522E2B29F
0x58f1223998f9ca900caf5a76370d6f16e882c81f05fd916d58768e35e9c1c0c0 0x4d380AEC34583E50f4E8DF9013160ac8e9b4E3fC
0xba42ce15368cc25ed4545b4a6a2cada74c2a46dd84c41c20b67204e389946c4f 0x1cfd117F377a23100081F1785ca53A4662B2b990
0xbdf10da3575e4431efe388ae75b03d03e854f460027982ce5152feae5add7110 0x3a1d32F39F901BB396eE877fD3A7C9Dbd0288D87
0x46eecb38

In [79]:
mev_bot_code = web3.eth.get_code("0x000000000035B5e5ad9019092C665357240f594e").hex()[2:]

In [80]:
# snippet 
# if (msg.data[0x04:0x24] >> 0xe0 == block.blockHash(block.number + ~0x00) >> 0xe0) { return; }
# (else branch) ... revert(memory[0x60:0xc4]);
c = 0
for i in range(0x04,0x24):
    c+=1
print(c)

32


In [81]:
mev_bot_code2 = web3.eth.get_code("0x0000000099cB7fC48a935BcEb9f05BbaE54e8987").hex()[2:]

In [82]:
print(mev_bot_code)

6004361061007f5760003560e01c80600f146100ec57806006146100e35780600b146100da578063fa461e33146100d157806009146100c857806002146100bf5780600e146100b657806003146100ad57806010146100a4578060111461009b57806012146100925780600114610089576004146100815761007f6100f5565b005b61007f610131565b5061007f610155565b5061007f6108ad565b5061007f6107b8565b5061007f610761565b5061007f6106ef565b5061007f61068b565b5061007f6105f9565b5061007f61053b565b5061007f6103f5565b5061007f61036c565b5061007f61030d565b5061007f61027d565b506060366000823760008136816f11239a45196deff0fbd62245a77b363f5af4903d9182610127575b1561012557f35bfd5b826000833e61011e565b5033733376ebc8dce3453a045a145ab7b1e728b2ed581e1461015257600080fd5b33ff5b61015d6109f8565b338073654fae4aa229d104cabead47e56703f58b174be41461027757807383b294466fe88cf8d4e73fd69b36e904d5db9133146102775780732a7424368f6a3deaaea212fa51b17d66aeb77a831461027757734a6feaaf06ed77421df557c6e213dd6997f759f714610272576101d1610c77565b60083560801c6000601960183560f81c610265575b826101f7929361020b575b5061

In [83]:
print(web3.eth.get_block(14010064))

AttributeDict({'baseFeePerGas': 104838506615, 'difficulty': 11605925319389407, 'extraData': HexBytes('0x457468657265756d50504c4e532f326d696e6572735f455532'), 'gasLimit': 30029266, 'gasUsed': 3621630, 'hash': HexBytes('0x9e5570dcd90179655a0af8e41b20c5b04613472dd8b4897d710da9f9b91c952f'), 'logsBloom': HexBytes('0x0020010000005001092000008000080240002040081000021001000001200180440000040000010004004100000005002300800108016001006008481021802000c00000010021988800000800000021000000004040000010100000c00014000a0000004600000304140016200888c0098800000200050002001010004c000000008104900090000248041480800080005002810108800986000040041040004208040401002000041012d00000100020200000080180000000020a9000000005200603000040102100010c000000904000000004844410002801020000a10040116008a000090001000500014108020040040020000150a008000040008421'), 'miner': '0x00192Fb10dF37c9FB26829eb2CC623cd1BF599E8', 'mixHash': HexBytes('0x5558b1add037b48d23c6514a05b9718214a966db72b14bb3f0e99ecd748014bc'), 'nonce': HexBytes('0xc1f6

In [84]:
# get the contracts with which our opal-like transactions are interacting
opal_opcode_txns_contracts = {}
for tx in opal_like_transactions:
    tx_to = web3_txns[tx["hash"]]['to']
    if tx_to not in opal_opcode_txns_contracts:
        opal_opcode_txns_contracts[tx_to] = []
    opal_opcode_txns_contracts[tx_to].append(tx)

In [85]:
if None in opal_opcode_txns_contracts:
    print(opal_opcode_txns_contracts[None])
    print(web3.eth.get_transaction("0x960663e787c6d4c6487765b9ac68f18bcb7667c74d11eed61ac5fcbe883cb9c8"))

In [86]:

opallike_sorted_contract_keys = []

tx_count = 0
for k in sorted(opal_opcode_txns_contracts, key=lambda k: len(opal_opcode_txns_contracts[k]), reverse=True):
    value = opal_opcode_txns_contracts[k]
    opallike_sorted_contract_keys.append(k)
    tx_count += len(value)
    print("https://etherscan.io/address/{0} had {1} interactions \n".format(k,len(value)))
    print("https://ethervm.io/decompile/{0}\n".format(k))
print("total tx count: "+str(tx_count))

https://etherscan.io/address/0x02BeeD1404c69e62b76Af6DbdaE41Bd98bcA2Eab had 1748 interactions 

https://ethervm.io/decompile/0x02BeeD1404c69e62b76Af6DbdaE41Bd98bcA2Eab

https://etherscan.io/address/0x5E4e65926BA27467555EB562121fac00D24E9dD2 had 1397 interactions 

https://ethervm.io/decompile/0x5E4e65926BA27467555EB562121fac00D24E9dD2

https://etherscan.io/address/0xdb7A53E6AE058E1Dcf4502341E2ADFA522E2B29F had 1351 interactions 

https://ethervm.io/decompile/0xdb7A53E6AE058E1Dcf4502341E2ADFA522E2B29F

https://etherscan.io/address/0x7CCB4EC695A2116E56A9F7b8738F78a15CD53bB0 had 459 interactions 

https://ethervm.io/decompile/0x7CCB4EC695A2116E56A9F7b8738F78a15CD53bB0

https://etherscan.io/address/0x000000000035B5e5ad9019092C665357240f594e had 432 interactions 

https://ethervm.io/decompile/0x000000000035B5e5ad9019092C665357240f594e

https://etherscan.io/address/0x00000000003b3cc22aF3aE1EAc0440BcEe416B40 had 242 interactions 

https://ethervm.io/decompile/0x00000000003b3cc22aF3aE1EAc0440B

In [125]:
for k in opallike_sorted_contract_keys:
    print(k)

0xc5F85281d4402850ff436b959a925a0e811D78d3
0x000000000035B5e5ad9019092C665357240f594e
0xEef86c2E49E11345F1a693675dF9a38f7d880C8F
0x5E4e65926BA27467555EB562121fac00D24E9dD2
0x56a76bcC92361f6DF8D75476feD8843EdC70e1C9
0xB6eD7644C69416d67B522e20bC294A9a9B405B31
0xd6e382aa7A09fc4A09C2fb99Cfce6A429985E65d
0x75E9Abc7E69fc46177d2F3538C0B92d89054eC91
0x563bDabAa8846ec445b25Bfbed88d160890a02Ed
0xa10FcA31A2Cb432C9Ac976779DC947CfDb003EF0
0x7DCDd6561B7FeA8c1C03dA9d605B36DA86AFe775
0x93a796B1E846567Fe3577af7B7BB89F71680173a
0x5650CA3f0289C762F83DdE1894faA9b6d0d89798
0x911605012f87A3017322c81fCB4C90ADA7C09116
0xd9506121D67fb918AC47AF0b883730694bE9377C
0x0000000099cB7fC48a935BcEb9f05BbaE54e8987
0x72519fa6cd095C99d5d67e31ddc117409Bc5c047
0x47e312d99C09Ce61A866c83cBbbbED5A4b9d33E7
0xB291984262259BcFe6Aa02b66a06e9769C5c1eF3
0xC84cC1111aDacE071e2a57a61c42450d4e133F16
0x7Be8076f4EA4A4AD08075C2508e481d6C946D12b
0x96c63f93B658213FF4ce366766DC915F1c1ECCcd
0x817b363f755e74Cee5eb4fd74db7A2638D43bDdF
0x619efFab7

In [188]:
import csv
#evm_opcodes = open("opallike_contract_analysis.csv", "r")
evm_opcodes = open("opallike_contract_analysis_new.csv", "r")

dict_reader = csv.DictReader(evm_opcodes)

print(dict_reader.fieldnames)
# dict to resolve opcode names to number
opallike_contract_dict = {}
for row in dict_reader:
    opallike_contract_dict[row["contract address"]]=row


['contract address', 'Opal-like tx interactions', 'estimated purpose', 'name', 'interesting', 'source code', 'inferred purpose for opcodes', '']


In [190]:
# make a dict with the interesting stuff?

print(opallike_contract_dict)

{'0x02BeeD1404c69e62b76Af6DbdaE41Bd98bcA2Eab': {'contract address': '0x02BeeD1404c69e62b76Af6DbdaE41Bd98bcA2Eab', 'Opal-like tx interactions': '1748', 'estimated purpose': 'NFT/Token', 'name': 'posers (pos)', 'interesting': '', 'source code': 'yes', 'inferred purpose for opcodes': 'randomness', '': 'Irandomizer.sol (owners provide impl address though)'}, '0xdb7A53E6AE058E1Dcf4502341E2ADFA522E2B29F': {'contract address': '0xdb7A53E6AE058E1Dcf4502341E2ADFA522E2B29F', 'Opal-like tx interactions': '579', 'estimated purpose': '?', 'name': '?', 'interesting': '', 'source code': 'no', 'inferred purpose for opcodes': '?', '': '        if (msg.data[0x04:0x24] >> 0xe0 == block.blockHash(block.number + ~0x00) >> 0xe0) { return; }\n    \n        memory[0x60:0x80] = 0x461bcd << 0xe5;\n        memory[0x64:0x84] = 0x20;\n        memory[0x84:0xa4] = 0x08;\n        memory[0xa4:0xc4] = 0x736176363a303031 << 0xc0;\n        revert(memory[0x60:0xc4]);'}, '0x5E4e65926BA27467555EB562121fac00D24E9dD2': {'cont

# Print some table with contract data

In [191]:
table = """
\\renewcommand{\\arraystretch}{1.3} % increase space between lines
\\begin{table}[!htpb]
\\centering
%\\scriptsize
\\resizebox{1.0\\linewidth}{!}{
\\begin{tabular}{c|c|c|p{2,5cm}|c|c}  \n"""


table += """\\textbf{Contract Address} & \\textbf{Num. TXns} & \\textbf{Purpose} & \\textbf{Name} & \\textbf{Source Code} & \\textbf{Opcode Purpose}  """


## fill in actual data

for key in opallike_contract_dict:
    table += """\\\\ \\hline
    """
    table += """ {} & {} & {} & {} & {} & {} \n""".format(
                                                "\\texttt{"+key+"}",
                                                opallike_contract_dict[key]["Opal-like tx interactions"],
                                                opallike_contract_dict[key]["estimated purpose"],
                                                opallike_contract_dict[key]["name"],
                                                opallike_contract_dict[key]["source code"],
                                                opallike_contract_dict[key]["inferred purpose for opcodes"])
    

table += """
\\end{tabular}
}
\\caption{Contracts with the highest number of \opal{}-like transaction interactions. Question marks (?) denote uncertain/unclear classifications.}
\\label{tab:opallikecontracts}
\\end{table}
"""

print(table)


\renewcommand{\arraystretch}{1.3} % increase space between lines
\begin{table}[!htpb]
\centering
%\scriptsize
\resizebox{1.0\linewidth}{!}{
\begin{tabular}{c|c|c|p{2,5cm}|c|c}  
\textbf{Contract Address} & \textbf{Num. TXns} & \textbf{Purpose} & \textbf{Name} & \textbf{Source Code} & \textbf{Opcode Purpose}  \\ \hline
     \texttt{0x02BeeD1404c69e62b76Af6DbdaE41Bd98bcA2Eab} & 1748 & NFT/Token & posers (pos) & yes & randomness 
\\ \hline
     \texttt{0xdb7A53E6AE058E1Dcf4502341E2ADFA522E2B29F} & 579 & ? & ? & no & ? 
\\ \hline
     \texttt{0x5E4e65926BA27467555EB562121fac00D24E9dD2} & 508 & Layer 2 rollup & optimism.io & yes & caching/processing 
\\ \hline
     \texttt{0x7CCB4EC695A2116E56A9F7b8738F78a15CD53bB0} & 444 & NFT/Token & Bright Blights (BRBL) & yes & ? 
\\ \hline
     \texttt{0x000000000035B5e5ad9019092C665357240f594e} & 196 & MEV Bot? & ? & no & context sensitivity? 
\\ \hline
     \texttt{0xB6eD7644C69416d67B522e20bC294A9a9B405B31} & 89 & Token & 0xbitcoin.org & yes & cont

In [129]:
print(opal_like_transactions[0])

{'hash': '0x5177299935e94f6f1f206ae27ea251b9c9e2ea3355625922884e4805d256b30e', 'interested_opcodes': {'NUMBER': 1, 'BLOCKHASH': 1}}


In [130]:
c = 0
with open(join(output_path,"opal_opcode_transactions.txt" ), 'wt+', encoding='utf-8') as f:
    for tx in opal_like_transactions:
        c+=1
        line = "https://etherscan.io/tx/{0} to address {1} \n".format(tx["hash"],web3_txns[tx["hash"]]['to'])
        f.write(line)
        print(line)
        
    f.write("Total count: {}".format(c))
    f.close()



https://etherscan.io/tx/0x5177299935e94f6f1f206ae27ea251b9c9e2ea3355625922884e4805d256b30e to address 0x5E4e65926BA27467555EB562121fac00D24E9dD2 

https://etherscan.io/tx/0x769c4147ef29768d32d8115fd84ef5d5c57f0290ebaefe9ca5d52a7793e2e472 to address 0xB6eD7644C69416d67B522e20bC294A9a9B405B31 

https://etherscan.io/tx/0x2d8e42143888da6ec9df80f8cb5fb23cda2ac9b141bce4fdcec6286b54782a0c to address 0x0000000099cB7fC48a935BcEb9f05BbaE54e8987 

https://etherscan.io/tx/0x6461c45c169f479011f5442137a53e5d868c0f1956e62b6434f2624a9ec65fe2 to address 0x5E4e65926BA27467555EB562121fac00D24E9dD2 

https://etherscan.io/tx/0xb52eca6490d91b75c7117e86019bd68f6c857ee49891a19b557a5595ea38d17e to address 0x0000000099cB7fC48a935BcEb9f05BbaE54e8987 

https://etherscan.io/tx/0xe94f3e51dae9c0db93ed3ebc188b662c245a106769dd37a1667cbbc8a397ff10 to address 0xd9506121D67fb918AC47AF0b883730694bE9377C 

https://etherscan.io/tx/0xd2719e081804ff4c9ddcabae5e00397f94e40385f06c88758de1f40b3edaeada to address 0x488C057CF3deE9

In [50]:
for tx in relevant_txns["0x241472827571aa34f5d799468578d0089b5897de49b4d20656534f8231cd7b2c"]:
    print(tx)

{'hash': '0x01bc6465f40824d0573696193f62b1567f0d21d5e3c3971ca5c5fa1619a249b5', 'failed': False, 'interested_opcodes': {'TIMESTAMP': 1}}
{'hash': '0x0478ad8d1b75cc3eaa321816b575a7159b86666c35c8f4b477d561fdd6ffb02c', 'failed': False, 'interested_opcodes': {'TIMESTAMP': 2}}
{'hash': '0x0adc6509e3cb01a31945df4602f96799ef8fed1b7efbaac3fa3193fc79332d6e', 'failed': False, 'interested_opcodes': {'TIMESTAMP': 1}}
{'hash': '0x10bbe13224be94867b1dcb6e4b01a077869c9c49fd9afcdd72116706c6eaa2cb', 'failed': False, 'interested_opcodes': {'TIMESTAMP': 3}}
{'hash': '0x15d17309ff8b74e5d3a5d7ab445871f7b6f3ba645005a6b72ac06cb6ae47c25c', 'failed': False, 'interested_opcodes': {'TIMESTAMP': 3}}
{'hash': '0x18552f68c19218f8c44af3339f87edb5a8525791004011a903b32a19f5d6c0fe', 'failed': True, 'interested_opcodes': {'NUMBER': 1}}
{'hash': '0x1de9782da47ab3f8438ae67e10d2153a239fbb4f83d4b785bdb800a669e0c03d', 'failed': False, 'interested_opcodes': {'TIMESTAMP': 3}}
{'hash': '0x25cded1a0b81cd094dd03a40a62d4ad81024f3ce

In [731]:
len(opcodes_blocks)

2936

In [52]:
for key,value in opcodes_blocks.items():
    print(key,value)

0x008e05e64954b7b91c70b78bb2a4ce6c7bf47e3fe99823a58dabe5cf6101d327 {'opcodes': defaultdict(<class 'int'>, {'CHAINID': 1, 'TIMESTAMP': 9, 'NUMBER': 1, 'SELFBALANCE': 2}), 'txcount': 69}
0x00a138204d130b91070e27bcdbb316c826416ea129c27b14760266e46c67e183 {'opcodes': defaultdict(<class 'int'>, {'TIMESTAMP': 4}), 'txcount': 20}
0x00fb5243c6ee225654ba51ed7a60a387617b35bc78a2e55ba7c78cfaea7159f1 {'opcodes': defaultdict(<class 'int'>, {'TIMESTAMP': 72, 'NUMBER': 15, 'SELFBALANCE': 19, 'BALANCE': 2}), 'txcount': 231}
0x010c577e33d6eb29ae3bc94279e6a5b2212a19e0bf8a5b58b8f0fea0d18f02e3 {'opcodes': defaultdict(<class 'int'>, {'BALANCE': 6, 'TIMESTAMP': 71, 'SELFBALANCE': 17, 'CHAINID': 1, 'NUMBER': 2}), 'txcount': 263}
0x01562a899609ca92eb1d7a0e3bff9d91972f0e605cb9f89dfd6059f0a567f9d3 {'opcodes': defaultdict(<class 'int'>, {'TIMESTAMP': 171, 'NUMBER': 3, 'BLOCKHASH': 2, 'COINBASE': 1, 'CHAINID': 1, 'SELFBALANCE': 1, 'BALANCE': 1}), 'txcount': 249}
0x018a49a018c0e4136b64e2ce4faf446baf37c378073b7280f

0xe59788f5ac84f31e2ee5a01f110bd791674d11c5954ab43bdf8d6a9ce2115ca4 {'opcodes': defaultdict(<class 'int'>, {'TIMESTAMP': 17, 'SELFBALANCE': 2, 'CHAINID': 1, 'NUMBER': 1}), 'txcount': 53}
0xe5a236044f1cf4d2c16ef63cad4fb0fc95e4507d4eb71b6ee3e47821f26e886f {'opcodes': defaultdict(<class 'int'>, {'TIMESTAMP': 24, 'SELFBALANCE': 7, 'NUMBER': 2}), 'txcount': 72}
0xe61c2b625d51f8e36716bd92773dd32e7a23bf97907a67c49e997d6e2683fb8f {'opcodes': defaultdict(<class 'int'>, {'TIMESTAMP': 49, 'NUMBER': 4, 'SELFBALANCE': 9, 'CHAINID': 2}), 'txcount': 235}
0xe66bbc0004d684b48d531d230485565914f52e08eb5c24f466de53665095f76f {'opcodes': defaultdict(<class 'int'>, {'TIMESTAMP': 12, 'NUMBER': 2, 'CHAINID': 1, 'SELFBALANCE': 1}), 'txcount': 52}
0xe66fba7e50a1770dadfdc3adec5cb0238d683877e93b47c782ab87b17cb58318 {'opcodes': defaultdict(<class 'int'>, {'TIMESTAMP': 73, 'SELFBALANCE': 19, 'BALANCE': 3, 'NUMBER': 10, 'BASEFEE': 1, 'COINBASE': 2, 'CHAINID': 1}), 'txcount': 539}
0xe67113f79d6308c550faaf9c66feed88b5b

In [732]:
print(getTotalTxcount())

545011


In [733]:


opcode_percentage = defaultdict(float)

# get opcode count for each block
for value in opcodes_blocks.values():
    # for each opcode we are interested in, add it to the counter
    for op in interested_opcodes:
        if op in value["opcodes"]:
            opcode_percentage[op] += value["opcodes"][op]
            
for key,value in opcode_percentage.items():
    opcode_percentage[key] /= getTotalTxcount()
        
print(opcode_percentage)

defaultdict(<class 'float'>, {'TIMESTAMP': 0.2160525200408799, 'NUMBER': 0.03852582791906952, 'BALANCE': 0.0040861560592355016, 'SELFBALANCE': 0.06376385063787704, 'CHAINID': 0.008047543994524882, 'BASEFEE': 0.0006587023014214392, 'BLOCKHASH': 0.0028201265662527912, 'COINBASE': 0.00269352361695452, 'DIFFICULTY': 0.0007889749014240079, 'GASLIMIT': 0.00019632631267992755})


ModuleNotFoundError: No module named 'pandas'

pc
op
gas
gasCost
depth
